In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import threading
import time
start_time = time.time()
#import datetime
#from datetime import timedelta, datetime
#import os


In [ ]:
ST_DETAILS = pd.read_csv('ST_DETAILS.csv',skip_blank_lines=True, engine="pyarrow")
MAJCAT_LIST = pd.read_csv('MAJCAT_LIST.csv',skip_blank_lines=True, engine="pyarrow")
ST_L_DETAILS = pd.read_csv('ST_L_DETAILS.csv',skip_blank_lines=True, engine="pyarrow")
BGT_MAJCAT_I = pd.read_csv('BGT_MAJCAT.csv',skip_blank_lines=True, engine="pyarrow")
BGT_MERCHCAT_I = pd.read_csv('BGT_MERCHCAT.csv',skip_blank_lines=True, engine="pyarrow")
ST_STK_I = pd.read_csv('ST_STK.csv',skip_blank_lines=True, engine="pyarrow")
DC_STK_I = pd.read_csv('DC_STK.csv',skip_blank_lines=True, engine="pyarrow")
MC_AUTO_ART_I = pd.read_csv('MC_AUTO_ART.csv',skip_blank_lines=True, engine="pyarrow")
MC_BODY_CONT_I = pd.read_csv('MC_BODY_CONT.csv',skip_blank_lines=True, engine="pyarrow")
MC_CLR_CONT_I = pd.read_csv('MC_CLR_CONT.csv',skip_blank_lines=True, engine="pyarrow")
MC_FABRIC_CONT_I = pd.read_csv('MC_FABRIC_CONT.csv',skip_blank_lines=True, engine="pyarrow")
MC_M_MVGR_CONT_I = pd.read_csv('MC_M_MVGR_CONT.csv',skip_blank_lines=True, engine="pyarrow")
MC_MVGR_CONT_I = pd.read_csv('MC_MVGR_CONT.csv',skip_blank_lines=True, engine="pyarrow")
MC_SEG_CONT_I = pd.read_csv('MC_SEG_CONT.csv',skip_blank_lines=True, engine="pyarrow")
MC_VEND_CONT_I = pd.read_csv('MC_VEND_CONT.csv',skip_blank_lines=True, engine="pyarrow")
ST_OPT_I = pd.read_csv('ST_OPT.csv',skip_blank_lines=True, engine="pyarrow")
OPT_DISP_Q_I = pd.read_csv('OPT_DISP_Q.csv',skip_blank_lines=True, engine="pyarrow")
ST_OPT_DISP_Q_I = pd.read_csv('ST_OPT_DISP_Q.csv',skip_blank_lines=True, engine="pyarrow")
ST_MC_CONS_I = pd.read_csv('ST_MC_CONS.csv',skip_blank_lines=True, engine="pyarrow")
ST_SSN_CONS_I = pd.read_csv('ST_SSN_CONS.csv',skip_blank_lines=True, engine="pyarrow")

In [ ]:
CAT = "M_JEANS"

# Filter each dataframe using .loc[]
BGT_MAJCAT = BGT_MAJCAT_I.loc[BGT_MAJCAT_I['MAJCAT'] == CAT].reset_index(drop=True)
BGT_MERCHCAT = BGT_MERCHCAT_I.loc[BGT_MERCHCAT_I['MAJCAT'] == CAT].reset_index(drop=True)
ST_STK = ST_STK_I.loc[ST_STK_I['MAJCAT'] == CAT].reset_index(drop=True)
DC_STK = DC_STK_I.loc[DC_STK_I['MAJCAT'] == CAT].reset_index(drop=True)
MC_AUTO_ART = MC_AUTO_ART_I.loc[MC_AUTO_ART_I['MAJCAT'] == CAT].reset_index(drop=True)
MC_BODY_CONT = MC_BODY_CONT_I.loc[MC_BODY_CONT_I['MAJCAT'] == CAT].reset_index(drop=True)
MC_CLR_CONT = MC_CLR_CONT_I.loc[MC_CLR_CONT_I['MAJCAT'] == CAT].reset_index(drop=True)
MC_FABRIC_CONT = MC_FABRIC_CONT_I.loc[MC_FABRIC_CONT_I['MAJCAT'] == CAT].reset_index(drop=True)
MC_M_MVGR_CONT = MC_M_MVGR_CONT_I.loc[MC_M_MVGR_CONT_I['MAJCAT'] == CAT].reset_index(drop=True)
MC_MVGR_CONT = MC_MVGR_CONT_I.loc[MC_MVGR_CONT_I['MAJCAT'] == CAT].reset_index(drop=True)
MC_SEG_CONT = MC_SEG_CONT_I.loc[MC_SEG_CONT_I['MAJCAT'] == CAT].reset_index(drop=True)
MC_VEND_CONT = MC_VEND_CONT_I.loc[MC_VEND_CONT_I['MAJCAT'] == CAT].reset_index(drop=True)
ST_OPT = ST_OPT_I.loc[ST_OPT_I['MAJCAT'] == CAT].reset_index(drop=True)
OPT_DISP_Q = OPT_DISP_Q_I.loc[OPT_DISP_Q_I['MAJCAT'] == CAT].reset_index(drop=True)
ST_OPT_DISP_Q = ST_OPT_DISP_Q_I.loc[ST_OPT_DISP_Q_I['MAJCAT'] == CAT].reset_index(drop=True)
ST_MC_CONS = ST_MC_CONS_I.loc[ST_MC_CONS_I['MAJCAT'] == CAT].reset_index(drop=True)
# ST_SSN_CONS = ST_SSN_CONS_I.loc[ST_SSN_CONS_I['MAJCAT'] == CAT].reset_index(drop=True)


In [ ]:
# converting column to object
ST_OPT_DISP_Q['ST_CD']=ST_OPT_DISP_Q['ST_CD'].astype(object)
# here inplace reduces copy operation
ST_DETAILS.sort_values(by='PRIORITY_LIST', ascending=True,inplace=True)


In [ ]:
# instead of deepCopy, i.e. copyByVAlues, we will copy by reference
MERCHCAT_MBQ = BGT_MERCHCAT.copy(deep=False)
MAJCAT_MBQ = BGT_MAJCAT.copy(deep=False)

MAJCAT_MBQ = pd.merge(MAJCAT_MBQ, ST_DETAILS, how="left",on=["ST_CD"],suffixes=('_left', '_right'))
print(MAJCAT_MBQ.columns.to_list())
MERCHCAT_MBQ = pd.merge(MERCHCAT_MBQ, ST_DETAILS, how="left",on=["ST_CD"],suffixes=('_left', '_right'))
MERCHCAT_MBQ = pd.merge(MERCHCAT_MBQ, ST_L_DETAILS, how="left",on=["ST_CD"],suffixes=('_left', '_right'))
MERCHCAT_MBQ = pd.merge(MERCHCAT_MBQ, OPT_DISP_Q, how="left",on=["MERCHCAT"],suffixes=('_left', '_right'))
MERCHCAT_MBQ = pd.merge(MERCHCAT_MBQ, ST_OPT_DISP_Q, how="left",on=["ST_CD","MERCHCAT"],suffixes=('_left', '_right'))

In [ ]:

MAJCAT_MBQ['REV_MJ_SALES_PD_1'] = (MAJCAT_MBQ['SALES_Q_1']*MAJCAT_MBQ['CM_RMN_SALES_CONS']+MAJCAT_MBQ['SALES_Q_2']*MAJCAT_MBQ['NM_RMN_SALES_CONS'])*(1/MAJCAT_MBQ['BGT_DAYS'])
MAJCAT_MBQ['MJ_MBQ_A'] = MAJCAT_MBQ['LISTING_M_1']*(MAJCAT_MBQ['ALLOC_DISP']+(MAJCAT_MBQ['REV_MJ_SALES_PD_1']*MAJCAT_MBQ['ALLOC_DAY_COVER']))
MAJCAT_MBQ['MJ_MBQ_30_1'] = MAJCAT_MBQ['LISTING_M_1']*(MAJCAT_MBQ['DISP_Q_1']+(MAJCAT_MBQ['REV_MJ_SALES_PD_1']*MAJCAT_MBQ['BGT_DAYS']))
MAJCAT_MBQ['REM_SALES_1'] = (MAJCAT_MBQ['SALES_Q_1']*MAJCAT_MBQ['CM_RMN_SALES_CONS'])*(1/MAJCAT_MBQ['RMN_BGT_DAYS'])
MAJCAT_MBQ['MJ_MBQ_30_2'] = MAJCAT_MBQ['LISTING_M_2']*(MAJCAT_MBQ['DISP_Q_2']+MAJCAT_MBQ['SALES_Q_2'])
MAJCAT_MBQ['MJ_MBQ_30_3'] = MAJCAT_MBQ['LISTING_M_3']*(MAJCAT_MBQ['DISP_Q_3']+MAJCAT_MBQ['SALES_Q_3'])
MAJCAT_MBQ['MJ_MBQ_30_4'] = MAJCAT_MBQ['LISTING_M_4']*(MAJCAT_MBQ['DISP_Q_4']+MAJCAT_MBQ['SALES_Q_4'])
MAJCAT_MBQ['MJ_MBQ_30_5'] = MAJCAT_MBQ['LISTING_M_5']*(MAJCAT_MBQ['DISP_Q_5']+MAJCAT_MBQ['SALES_Q_5'])
MAJCAT_MBQ['MJ_MBQ_30_6'] = MAJCAT_MBQ['LISTING_M_6']*(MAJCAT_MBQ['DISP_Q_6']+MAJCAT_MBQ['SALES_Q_6'])
MAJCAT_MBQ['MJ_MBQ_30_7'] = MAJCAT_MBQ['LISTING_M_7']*(MAJCAT_MBQ['DISP_Q_7']+MAJCAT_MBQ['SALES_Q_7'])
MAJCAT_MBQ['MJ_MBQ_30_8'] = MAJCAT_MBQ['LISTING_M_8']*(MAJCAT_MBQ['DISP_Q_8']+MAJCAT_MBQ['SALES_Q_8'])
MAJCAT_MBQ['MJ_MBQ_30_9'] = MAJCAT_MBQ['LISTING_M_9']*(MAJCAT_MBQ['DISP_Q_9']+MAJCAT_MBQ['SALES_Q_9'])
MAJCAT_MBQ['MJ_MBQ_30_10'] = MAJCAT_MBQ['LISTING_M_10']*(MAJCAT_MBQ['DISP_Q_10']+MAJCAT_MBQ['SALES_Q_10'])
MAJCAT_MBQ['MJ_MBQ_30_11'] = MAJCAT_MBQ['LISTING_M_11']*(MAJCAT_MBQ['DISP_Q_11']+MAJCAT_MBQ['SALES_Q_11'])
MAJCAT_MBQ['MJ_MBQ_30_12'] = MAJCAT_MBQ['LISTING_M_12']*(MAJCAT_MBQ['DISP_Q_12']+MAJCAT_MBQ['SALES_Q_12'])

In [ ]:
MERCHCAT_MBQ['F_OPT_DISP_Q_1'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_1'] + 0 > 0, 
                                          MERCHCAT_MBQ['ST_OPT_DISP_Q_1'], 
                                          MERCHCAT_MBQ['OPT_DISP_Q_1'])

MERCHCAT_MBQ['F_OPT_DISP_Q_2'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_2'] + 0 > 0, 
                                          MERCHCAT_MBQ['ST_OPT_DISP_Q_2'], 
                                          MERCHCAT_MBQ['OPT_DISP_Q_2'])

MERCHCAT_MBQ['F_OPT_DISP_Q_3'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_3'] + 0 > 0, 
                                          MERCHCAT_MBQ['ST_OPT_DISP_Q_3'], 
                                          MERCHCAT_MBQ['OPT_DISP_Q_3'])

MERCHCAT_MBQ['F_OPT_DISP_Q_4'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_4'] + 0 > 0, 
                                          MERCHCAT_MBQ['ST_OPT_DISP_Q_4'], 
                                          MERCHCAT_MBQ['OPT_DISP_Q_4'])

MERCHCAT_MBQ['F_OPT_DISP_Q_5'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_5'] + 0 > 0, 
                                          MERCHCAT_MBQ['ST_OPT_DISP_Q_5'], 
                                          MERCHCAT_MBQ['OPT_DISP_Q_5'])

MERCHCAT_MBQ['F_OPT_DISP_Q_6'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_6'] + 0 > 0, 
                                          MERCHCAT_MBQ['ST_OPT_DISP_Q_6'], 
                                          MERCHCAT_MBQ['OPT_DISP_Q_6'])

MERCHCAT_MBQ['F_OPT_DISP_Q_7'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_7'] + 0 > 0, 
                                          MERCHCAT_MBQ['ST_OPT_DISP_Q_7'], 
                                          MERCHCAT_MBQ['OPT_DISP_Q_7'])

MERCHCAT_MBQ['F_OPT_DISP_Q_8'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_8'] + 0 > 0, 
                                          MERCHCAT_MBQ['ST_OPT_DISP_Q_8'], 
                                          MERCHCAT_MBQ['OPT_DISP_Q_8'])

MERCHCAT_MBQ['F_OPT_DISP_Q_9'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_9'] + 0 > 0, 
                                          MERCHCAT_MBQ['ST_OPT_DISP_Q_9'], 
                                          MERCHCAT_MBQ['OPT_DISP_Q_9'])

MERCHCAT_MBQ['F_OPT_DISP_Q_10'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_10'] + 0 > 0, 
                                           MERCHCAT_MBQ['ST_OPT_DISP_Q_10'], 
                                           MERCHCAT_MBQ['OPT_DISP_Q_10'])

MERCHCAT_MBQ['F_OPT_DISP_Q_11'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_11'] + 0 > 0, 
                                           MERCHCAT_MBQ['ST_OPT_DISP_Q_11'], 
                                           MERCHCAT_MBQ['OPT_DISP_Q_11'])

MERCHCAT_MBQ['F_OPT_DISP_Q_12'] = np.where(MERCHCAT_MBQ['ST_OPT_DISP_Q_12'] + 0 > 0, 
                                           MERCHCAT_MBQ['ST_OPT_DISP_Q_12'], 
                                           MERCHCAT_MBQ['OPT_DISP_Q_12'])


In [ ]:
#OPT_CNT FINALISATION

MERCHCAT_MBQ['OPT_CNT_A'] = np.where((MERCHCAT_MBQ['ALLOC_DISP'] / MERCHCAT_MBQ['F_OPT_DISP_Q_1']) + 0 > 0,
                                         np.round(MERCHCAT_MBQ['ALLOC_DISP'] / MERCHCAT_MBQ['F_OPT_DISP_Q_1']),
                                         0)

MERCHCAT_MBQ['OPT_CNT_1'] = np.where((MERCHCAT_MBQ['DISP_Q_1']/MERCHCAT_MBQ['F_OPT_DISP_Q_1'])+0>0,
                                         np.round(MERCHCAT_MBQ['DISP_Q_1']/MERCHCAT_MBQ['F_OPT_DISP_Q_1']),
                                         0)

# OPT_CNT_2
MERCHCAT_MBQ['OPT_CNT_2'] = np.where((MERCHCAT_MBQ['DISP_Q_2'] / MERCHCAT_MBQ['F_OPT_DISP_Q_2']) + 0 > 0,
                                      np.round(MERCHCAT_MBQ['DISP_Q_2'] / MERCHCAT_MBQ['F_OPT_DISP_Q_2']),
                                      0)

# OPT_CNT_3
MERCHCAT_MBQ['OPT_CNT_3'] = np.where((MERCHCAT_MBQ['DISP_Q_3'] / MERCHCAT_MBQ['F_OPT_DISP_Q_3']) + 0 > 0,
                                      np.round(MERCHCAT_MBQ['DISP_Q_3'] / MERCHCAT_MBQ['F_OPT_DISP_Q_3']),
                                      0)

# OPT_CNT_4
MERCHCAT_MBQ['OPT_CNT_4'] = np.where((MERCHCAT_MBQ['DISP_Q_4'] / MERCHCAT_MBQ['F_OPT_DISP_Q_4']) + 0 > 0,
                                      np.round(MERCHCAT_MBQ['DISP_Q_4'] / MERCHCAT_MBQ['F_OPT_DISP_Q_4']),
                                      0)

# OPT_CNT_5
MERCHCAT_MBQ['OPT_CNT_5'] = np.where((MERCHCAT_MBQ['DISP_Q_5'] / MERCHCAT_MBQ['F_OPT_DISP_Q_5']) + 0 > 0,
                                      np.round(MERCHCAT_MBQ['DISP_Q_5'] / MERCHCAT_MBQ['F_OPT_DISP_Q_5']),
                                      0)

# OPT_CNT_6
MERCHCAT_MBQ['OPT_CNT_6'] = np.where((MERCHCAT_MBQ['DISP_Q_6'] / MERCHCAT_MBQ['F_OPT_DISP_Q_6']) + 0 > 0,
                                      np.round(MERCHCAT_MBQ['DISP_Q_6'] / MERCHCAT_MBQ['F_OPT_DISP_Q_6']),
                                      0)

# OPT_CNT_7
MERCHCAT_MBQ['OPT_CNT_7'] = np.where((MERCHCAT_MBQ['DISP_Q_7'] / MERCHCAT_MBQ['F_OPT_DISP_Q_7']) + 0 > 0,
                                      np.round(MERCHCAT_MBQ['DISP_Q_7'] / MERCHCAT_MBQ['F_OPT_DISP_Q_7']),
                                      0)

# OPT_CNT_8
MERCHCAT_MBQ['OPT_CNT_8'] = np.where((MERCHCAT_MBQ['DISP_Q_8'] / MERCHCAT_MBQ['F_OPT_DISP_Q_8']) + 0 > 0,
                                      np.round(MERCHCAT_MBQ['DISP_Q_8'] / MERCHCAT_MBQ['F_OPT_DISP_Q_8']),
                                      0)

# OPT_CNT_9
MERCHCAT_MBQ['OPT_CNT_9'] = np.where((MERCHCAT_MBQ['DISP_Q_9'] / MERCHCAT_MBQ['F_OPT_DISP_Q_9']) + 0 > 0,
                                      np.round(MERCHCAT_MBQ['DISP_Q_9'] / MERCHCAT_MBQ['F_OPT_DISP_Q_9']),
                                      0)

# OPT_CNT_10
MERCHCAT_MBQ['OPT_CNT_10'] = np.where((MERCHCAT_MBQ['DISP_Q_10'] / MERCHCAT_MBQ['F_OPT_DISP_Q_10']) + 0 > 0,
                                       np.round(MERCHCAT_MBQ['DISP_Q_10'] / MERCHCAT_MBQ['F_OPT_DISP_Q_10']),
                                       0)

# OPT_CNT_11
MERCHCAT_MBQ['OPT_CNT_11'] = np.where((MERCHCAT_MBQ['DISP_Q_11'] / MERCHCAT_MBQ['F_OPT_DISP_Q_11']) + 0 > 0,
                                       np.round(MERCHCAT_MBQ['DISP_Q_11'] / MERCHCAT_MBQ['F_OPT_DISP_Q_11']),
                                       0)

# OPT_CNT_12
MERCHCAT_MBQ['OPT_CNT_12'] = np.where((MERCHCAT_MBQ['DISP_Q_12'] / MERCHCAT_MBQ['F_OPT_DISP_Q_12']) + 0 > 0,
                                       np.round(MERCHCAT_MBQ['DISP_Q_12'] / MERCHCAT_MBQ['F_OPT_DISP_Q_12']),
                                       0)


In [ ]:
MERCHCAT_MBQ['OPT_SALES_A'] = np.where((MERCHCAT_MBQ['SALES_Q_1'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_A'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_1'] / MERCHCAT_MBQ['OPT_CNT_A'], 2))

MERCHCAT_MBQ['OPT_SALES_1'] = np.where((MERCHCAT_MBQ['SALES_Q_1'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_1'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_1'] / MERCHCAT_MBQ['OPT_CNT_1'], 2))

MERCHCAT_MBQ['OPT_SALES_2'] = np.where((MERCHCAT_MBQ['SALES_Q_2'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_2'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_2'] / MERCHCAT_MBQ['OPT_CNT_2'], 2))

MERCHCAT_MBQ['OPT_SALES_3'] = np.where((MERCHCAT_MBQ['SALES_Q_3'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_3'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_3'] / MERCHCAT_MBQ['OPT_CNT_3'], 2))

MERCHCAT_MBQ['OPT_SALES_4'] = np.where((MERCHCAT_MBQ['SALES_Q_4'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_4'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_4'] / MERCHCAT_MBQ['OPT_CNT_4'], 2))

MERCHCAT_MBQ['OPT_SALES_5'] = np.where((MERCHCAT_MBQ['SALES_Q_5'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_5'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_5'] / MERCHCAT_MBQ['OPT_CNT_5'], 2))

MERCHCAT_MBQ['OPT_SALES_6'] = np.where((MERCHCAT_MBQ['SALES_Q_6'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_6'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_6'] / MERCHCAT_MBQ['OPT_CNT_6'], 2))

MERCHCAT_MBQ['OPT_SALES_7'] = np.where((MERCHCAT_MBQ['SALES_Q_7'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_7'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_7'] / MERCHCAT_MBQ['OPT_CNT_7'], 2))

MERCHCAT_MBQ['OPT_SALES_8'] = np.where((MERCHCAT_MBQ['SALES_Q_8'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_8'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_8'] / MERCHCAT_MBQ['OPT_CNT_8'], 2))

MERCHCAT_MBQ['OPT_SALES_9'] = np.where((MERCHCAT_MBQ['SALES_Q_9'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_9'] + 0 == 0),
                                       0,
                                       np.round(MERCHCAT_MBQ['SALES_Q_9'] / MERCHCAT_MBQ['OPT_CNT_9'], 2))

MERCHCAT_MBQ['OPT_SALES_10'] = np.where((MERCHCAT_MBQ['SALES_Q_10'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_10'] + 0 == 0),
                                        0,
                                        np.round(MERCHCAT_MBQ['SALES_Q_10'] / MERCHCAT_MBQ['OPT_CNT_10'], 2))

MERCHCAT_MBQ['OPT_SALES_11'] = np.where((MERCHCAT_MBQ['SALES_Q_11'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_11'] + 0 == 0),
                                        0,
                                        np.round(MERCHCAT_MBQ['SALES_Q_11'] / MERCHCAT_MBQ['OPT_CNT_11'], 2))

MERCHCAT_MBQ['OPT_SALES_12'] = np.where((MERCHCAT_MBQ['SALES_Q_12'] + 0 == 0) | (MERCHCAT_MBQ['OPT_CNT_12'] + 0 == 0),
                                        0,
                                        np.round(MERCHCAT_MBQ['SALES_Q_12'] / MERCHCAT_MBQ['OPT_CNT_12'], 2))


In [ ]:
#ALLOC MBQ
MERCHCAT_MBQ['REV_OPT_SALES_PD_A'] = (MERCHCAT_MBQ['OPT_SALES_A']*MERCHCAT_MBQ['CM_RMN_SALES_CONS']+MERCHCAT_MBQ['OPT_SALES_2']*MERCHCAT_MBQ['NM_RMN_SALES_CONS'])*(1/MERCHCAT_MBQ['BGT_DAYS'])
# Calculate REV_OPT_SALES_PD_1
MERCHCAT_MBQ['REV_OPT_SALES_PD_1'] = ((MERCHCAT_MBQ['OPT_SALES_1'] * MERCHCAT_MBQ['CM_RMN_SALES_CONS']) +
                                      (MERCHCAT_MBQ['OPT_SALES_2'] * MERCHCAT_MBQ['NM_RMN_SALES_CONS'])) * (1 / MERCHCAT_MBQ['BGT_DAYS'])

# Calculate OPT_MBQ_30_1
MERCHCAT_MBQ['OPT_MBQ_30_1'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_1'] + (MERCHCAT_MBQ['REV_OPT_SALES_PD_1'] * MERCHCAT_MBQ['BGT_DAYS']))

# Calculate REM_OPT_SALES_PD_1
MERCHCAT_MBQ['REM_OPT_SALES_PD_1'] = (MERCHCAT_MBQ['OPT_SALES_1'] * MERCHCAT_MBQ['CM_RMN_SALES_CONS']) * (1 / MERCHCAT_MBQ['RMN_BGT_DAYS'])

# Calculate OPT_MBQ_A
MERCHCAT_MBQ['OPT_MBQ_A'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_1'] + (MERCHCAT_MBQ['REV_OPT_SALES_PD_1'] * MERCHCAT_MBQ['ALLOC_DAY_COVER']))


In [ ]:
# Calculate OPT_MBQ_30_2 to OPT_MBQ_30_12
MERCHCAT_MBQ['OPT_MBQ_30_2'] = MERCHCAT_MBQ['LISTING_M_2'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_2'] + MERCHCAT_MBQ['OPT_SALES_2'])
MERCHCAT_MBQ['OPT_MBQ_30_3'] = MERCHCAT_MBQ['LISTING_M_3'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_3'] + MERCHCAT_MBQ['OPT_SALES_3'])
MERCHCAT_MBQ['OPT_MBQ_30_4'] = MERCHCAT_MBQ['LISTING_M_4'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_4'] + MERCHCAT_MBQ['OPT_SALES_4'])
MERCHCAT_MBQ['OPT_MBQ_30_5'] = MERCHCAT_MBQ['LISTING_M_5'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_5'] + MERCHCAT_MBQ['OPT_SALES_5'])
MERCHCAT_MBQ['OPT_MBQ_30_6'] = MERCHCAT_MBQ['LISTING_M_6'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_6'] + MERCHCAT_MBQ['OPT_SALES_6'])
MERCHCAT_MBQ['OPT_MBQ_30_7'] = MERCHCAT_MBQ['LISTING_M_7'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_7'] + MERCHCAT_MBQ['OPT_SALES_7'])
MERCHCAT_MBQ['OPT_MBQ_30_8'] = MERCHCAT_MBQ['LISTING_M_8'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_8'] + MERCHCAT_MBQ['OPT_SALES_8'])
MERCHCAT_MBQ['OPT_MBQ_30_9'] = MERCHCAT_MBQ['LISTING_M_9'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_9'] + MERCHCAT_MBQ['OPT_SALES_9'])
MERCHCAT_MBQ['OPT_MBQ_30_10'] = MERCHCAT_MBQ['LISTING_M_10'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_10'] + MERCHCAT_MBQ['OPT_SALES_10'])
MERCHCAT_MBQ['OPT_MBQ_30_11'] = MERCHCAT_MBQ['LISTING_M_11'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_11'] + MERCHCAT_MBQ['OPT_SALES_11'])
MERCHCAT_MBQ['OPT_MBQ_30_12'] = MERCHCAT_MBQ['LISTING_M_12'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_12'] + MERCHCAT_MBQ['OPT_SALES_12'])


In [ ]:
# Calculate OPT_MBQ_L1_A and OPT_MBQ_L1_1
MERCHCAT_MBQ['OPT_MBQ_L1_A'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_1'] + (MERCHCAT_MBQ['REV_OPT_SALES_PD_A'] * MERCHCAT_MBQ['L1_SALES_COVER']))
MERCHCAT_MBQ['OPT_MBQ_L1_1'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_1'] + (MERCHCAT_MBQ['REV_OPT_SALES_PD_1'] * MERCHCAT_MBQ['L1_SALES_COVER']))

# Calculate OPT_MBQ_L1_2
MERCHCAT_MBQ['OPT_MBQ_L1_2'] = MERCHCAT_MBQ['LISTING_M_2'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_2'] + (MERCHCAT_MBQ['OPT_SALES_2'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_3
MERCHCAT_MBQ['OPT_MBQ_L1_3'] = MERCHCAT_MBQ['LISTING_M_3'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_3'] + (MERCHCAT_MBQ['OPT_SALES_3'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_4
MERCHCAT_MBQ['OPT_MBQ_L1_4'] = MERCHCAT_MBQ['LISTING_M_4'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_4'] + (MERCHCAT_MBQ['OPT_SALES_4'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_5
MERCHCAT_MBQ['OPT_MBQ_L1_5'] = MERCHCAT_MBQ['LISTING_M_5'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_5'] + (MERCHCAT_MBQ['OPT_SALES_5'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_6
MERCHCAT_MBQ['OPT_MBQ_L1_6'] = MERCHCAT_MBQ['LISTING_M_6'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_6'] + (MERCHCAT_MBQ['OPT_SALES_6'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_7
MERCHCAT_MBQ['OPT_MBQ_L1_7'] = MERCHCAT_MBQ['LISTING_M_7'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_7'] + (MERCHCAT_MBQ['OPT_SALES_7'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_8
MERCHCAT_MBQ['OPT_MBQ_L1_8'] = MERCHCAT_MBQ['LISTING_M_8'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_8'] + (MERCHCAT_MBQ['OPT_SALES_8'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_9
MERCHCAT_MBQ['OPT_MBQ_L1_9'] = MERCHCAT_MBQ['LISTING_M_9'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_9'] + (MERCHCAT_MBQ['OPT_SALES_9'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_10
MERCHCAT_MBQ['OPT_MBQ_L1_10'] = MERCHCAT_MBQ['LISTING_M_10'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_10'] + (MERCHCAT_MBQ['OPT_SALES_10'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_11
MERCHCAT_MBQ['OPT_MBQ_L1_11'] = MERCHCAT_MBQ['LISTING_M_11'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_11'] + (MERCHCAT_MBQ['OPT_SALES_11'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))

# Calculate OPT_MBQ_L1_12
MERCHCAT_MBQ['OPT_MBQ_L1_12'] = MERCHCAT_MBQ['LISTING_M_12'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_12'] + (MERCHCAT_MBQ['OPT_SALES_12'] * (MERCHCAT_MBQ['L1_SALES_COVER'] / 30)))


In [ ]:
# Calculate OPT_MBQ_30_A
MERCHCAT_MBQ['OPT_MBQ_30_A'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_1'] + (MERCHCAT_MBQ['REV_OPT_SALES_PD_A'] * MERCHCAT_MBQ['ALLOC_DAY_COVER'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_1']))

# Calculate OPT_MBQ_30_1
MERCHCAT_MBQ['OPT_MBQ_30_1'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_1'] + (MERCHCAT_MBQ['REV_OPT_SALES_PD_1'] * MERCHCAT_MBQ['BGT_DAYS'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_1']))

# Calculate OPT_MBQ_30_2
MERCHCAT_MBQ['OPT_MBQ_30_2'] = MERCHCAT_MBQ['LISTING_M_2'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_2'] + (MERCHCAT_MBQ['OPT_SALES_2'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_2']))

# Calculate OPT_MBQ_30_3
MERCHCAT_MBQ['OPT_MBQ_30_3'] = MERCHCAT_MBQ['LISTING_M_3'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_3'] + (MERCHCAT_MBQ['OPT_SALES_3'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_3']))

# Calculate OPT_MBQ_30_4
MERCHCAT_MBQ['OPT_MBQ_30_4'] = MERCHCAT_MBQ['LISTING_M_4'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_4'] + (MERCHCAT_MBQ['OPT_SALES_4'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_4']))

# Calculate OPT_MBQ_30_5
MERCHCAT_MBQ['OPT_MBQ_30_5'] = MERCHCAT_MBQ['LISTING_M_5'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_5'] + (MERCHCAT_MBQ['OPT_SALES_5'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_5']))

# Calculate OPT_MBQ_30_6
MERCHCAT_MBQ['OPT_MBQ_30_6'] = MERCHCAT_MBQ['LISTING_M_6'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_6'] + (MERCHCAT_MBQ['OPT_SALES_6'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_6']))

# Calculate OPT_MBQ_30_7
MERCHCAT_MBQ['OPT_MBQ_30_7'] = MERCHCAT_MBQ['LISTING_M_7'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_7'] + (MERCHCAT_MBQ['OPT_SALES_7'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_7']))

# Calculate OPT_MBQ_30_8
MERCHCAT_MBQ['OPT_MBQ_30_8'] = MERCHCAT_MBQ['LISTING_M_8'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_8'] + (MERCHCAT_MBQ['OPT_SALES_8'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_8']))

# Calculate OPT_MBQ_30_9
MERCHCAT_MBQ['OPT_MBQ_30_9'] = MERCHCAT_MBQ['LISTING_M_9'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_9'] + (MERCHCAT_MBQ['OPT_SALES_9'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_9']))

# Calculate OPT_MBQ_30_10
MERCHCAT_MBQ['OPT_MBQ_30_10'] = MERCHCAT_MBQ['LISTING_M_10'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_10'] + (MERCHCAT_MBQ['OPT_SALES_10'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_10']))

# Calculate OPT_MBQ_30_11
MERCHCAT_MBQ['OPT_MBQ_30_11'] = MERCHCAT_MBQ['LISTING_M_11'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_11'] + (MERCHCAT_MBQ['OPT_SALES_11'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_11']))

# Calculate OPT_MBQ_30_12
MERCHCAT_MBQ['OPT_MBQ_30_12'] = MERCHCAT_MBQ['LISTING_M_12'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_12'] + (MERCHCAT_MBQ['OPT_SALES_12'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_12']))


In [ ]:
# Calculate SSN_MBQ_A
MERCHCAT_MBQ['SSN_MBQ_A'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_1'] + (MERCHCAT_MBQ['REV_OPT_SALES_PD_A'] * MERCHCAT_MBQ['ALLOC_DAY_COVER'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_1']))

# Calculate SSN_MBQ_1
MERCHCAT_MBQ['SSN_MBQ_1'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_1'] + (MERCHCAT_MBQ['REV_OPT_SALES_PD_1'] * MERCHCAT_MBQ['BGT_DAYS'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_1']))
    
# Calculate SSN_MBQ_2
MERCHCAT_MBQ['SSN_MBQ_2'] = MERCHCAT_MBQ['LISTING_M_2'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_2'] + (MERCHCAT_MBQ['OPT_SALES_2'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_2']))

# Calculate SSN_MBQ_3
MERCHCAT_MBQ['SSN_MBQ_3'] = MERCHCAT_MBQ['LISTING_M_3'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_3'] + (MERCHCAT_MBQ['OPT_SALES_3'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_3']))

# Calculate SSN_MBQ_4
MERCHCAT_MBQ['SSN_MBQ_4'] = MERCHCAT_MBQ['LISTING_M_4'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_4'] + (MERCHCAT_MBQ['OPT_SALES_4'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_4']))

# Calculate SSN_MBQ_5
MERCHCAT_MBQ['SSN_MBQ_5'] = MERCHCAT_MBQ['LISTING_M_5'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_5'] + (MERCHCAT_MBQ['OPT_SALES_5'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_5']))

# Calculate SSN_MBQ_6
MERCHCAT_MBQ['SSN_MBQ_6'] = MERCHCAT_MBQ['LISTING_M_6'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_6'] + (MERCHCAT_MBQ['OPT_SALES_6'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_6']))

# Calculate SSN_MBQ_7
MERCHCAT_MBQ['SSN_MBQ_7'] = MERCHCAT_MBQ['LISTING_M_7'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_7'] + (MERCHCAT_MBQ['OPT_SALES_7'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_7']))

# Calculate SSN_MBQ_8
MERCHCAT_MBQ['SSN_MBQ_8'] = MERCHCAT_MBQ['LISTING_M_8'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_8'] + (MERCHCAT_MBQ['OPT_SALES_8'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_8']))

# Calculate SSN_MBQ_9
MERCHCAT_MBQ['SSN_MBQ_9'] = MERCHCAT_MBQ['LISTING_M_9'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_9'] + (MERCHCAT_MBQ['OPT_SALES_9'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_9']))

# Calculate SSN_MBQ_10
MERCHCAT_MBQ['SSN_MBQ_10'] = MERCHCAT_MBQ['LISTING_M_10'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_10'] + (MERCHCAT_MBQ['OPT_SALES_10'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_10']))

# Calculate SSN_MBQ_11
MERCHCAT_MBQ['SSN_MBQ_11'] = MERCHCAT_MBQ['LISTING_M_11'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_11'] + (MERCHCAT_MBQ['OPT_SALES_11'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_11']))

# Calculate SSN_MBQ_12
MERCHCAT_MBQ['SSN_MBQ_12'] = MERCHCAT_MBQ['LISTING_M_12'] * (MERCHCAT_MBQ['F_OPT_DISP_Q_12'] + (MERCHCAT_MBQ['OPT_SALES_12'] * MERCHCAT_MBQ['L_OPT_SALES_GR_M_12']))


In [ ]:
# Calculate REV_MC_SALES_PD_1
MERCHCAT_MBQ['REV_MC_SALES_PD_1'] = (MERCHCAT_MBQ['SALES_Q_1'] * MERCHCAT_MBQ['CM_RMN_SALES_CONS'] + MERCHCAT_MBQ['SALES_Q_2'] * MERCHCAT_MBQ['NM_RMN_SALES_CONS']) * (1 / MERCHCAT_MBQ['BGT_DAYS'])

# Calculate MC_MBQ_A
MERCHCAT_MBQ['MC_MBQ_A'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['ALLOC_DISP'] + (MERCHCAT_MBQ['REV_MC_SALES_PD_1'] * MERCHCAT_MBQ['ALLOC_DAY_COVER']))

# Calculate MC_MBQ_30_1
MERCHCAT_MBQ['MC_MBQ_30_1'] = MERCHCAT_MBQ['LISTING_M_1'] * (MERCHCAT_MBQ['DISP_Q_1'] + (MERCHCAT_MBQ['REV_MC_SALES_PD_1'] * MERCHCAT_MBQ['BGT_DAYS']))

# Calculate REM_SALES_1
MERCHCAT_MBQ['REM_SALES_1'] = (MERCHCAT_MBQ['SALES_Q_1'] * MERCHCAT_MBQ['CM_RMN_SALES_CONS']) * (1 / MERCHCAT_MBQ['RMN_BGT_DAYS'])


In [ ]:
# Calculate MC_MBQ_30_2
MERCHCAT_MBQ['MC_MBQ_30_2'] = MERCHCAT_MBQ['LISTING_M_2'] * (MERCHCAT_MBQ['DISP_Q_2'] + MERCHCAT_MBQ['SALES_Q_2'])

# Calculate MC_MBQ_30_3
MERCHCAT_MBQ['MC_MBQ_30_3'] = MERCHCAT_MBQ['LISTING_M_3'] * (MERCHCAT_MBQ['DISP_Q_3'] + MERCHCAT_MBQ['SALES_Q_3'])

# Calculate MC_MBQ_30_4
MERCHCAT_MBQ['MC_MBQ_30_4'] = MERCHCAT_MBQ['LISTING_M_4'] * (MERCHCAT_MBQ['DISP_Q_4'] + MERCHCAT_MBQ['SALES_Q_4'])

# Calculate MC_MBQ_30_5
MERCHCAT_MBQ['MC_MBQ_30_5'] = MERCHCAT_MBQ['LISTING_M_5'] * (MERCHCAT_MBQ['DISP_Q_5'] + MERCHCAT_MBQ['SALES_Q_5'])

# Calculate MC_MBQ_30_6
MERCHCAT_MBQ['MC_MBQ_30_6'] = MERCHCAT_MBQ['LISTING_M_6'] * (MERCHCAT_MBQ['DISP_Q_6'] + MERCHCAT_MBQ['SALES_Q_6'])

# Calculate MC_MBQ_30_7
MERCHCAT_MBQ['MC_MBQ_30_7'] = MERCHCAT_MBQ['LISTING_M_7'] * (MERCHCAT_MBQ['DISP_Q_7'] + MERCHCAT_MBQ['SALES_Q_7'])

# Calculate MC_MBQ_30_8
MERCHCAT_MBQ['MC_MBQ_30_8'] = MERCHCAT_MBQ['LISTING_M_8'] * (MERCHCAT_MBQ['DISP_Q_8'] + MERCHCAT_MBQ['SALES_Q_8'])

# Calculate MC_MBQ_30_9
MERCHCAT_MBQ['MC_MBQ_30_9'] = MERCHCAT_MBQ['LISTING_M_9'] * (MERCHCAT_MBQ['DISP_Q_9'] + MERCHCAT_MBQ['SALES_Q_9'])

# Calculate MC_MBQ_30_10
MERCHCAT_MBQ['MC_MBQ_30_10'] = MERCHCAT_MBQ['LISTING_M_10'] * (MERCHCAT_MBQ['DISP_Q_10'] + MERCHCAT_MBQ['SALES_Q_10'])

# Calculate MC_MBQ_30_11
MERCHCAT_MBQ['MC_MBQ_30_11'] = MERCHCAT_MBQ['LISTING_M_11'] * (MERCHCAT_MBQ['DISP_Q_11'] + MERCHCAT_MBQ['SALES_Q_11'])

# Calculate MC_MBQ_30_12
MERCHCAT_MBQ['MC_MBQ_30_12'] = MERCHCAT_MBQ['LISTING_M_12'] * (MERCHCAT_MBQ['DISP_Q_12'] + MERCHCAT_MBQ['SALES_Q_12'])


In [ ]:
MERCHCAT_MBQ.drop(['OPT_DISP_Q_1','OPT_DISP_Q_2','OPT_DISP_Q_3','OPT_DISP_Q_4','OPT_DISP_Q_5','OPT_DISP_Q_6'],inplace=True,axis=1)
MERCHCAT_MBQ.drop(['OPT_DISP_Q_7','OPT_DISP_Q_8','OPT_DISP_Q_9','OPT_DISP_Q_10','OPT_DISP_Q_11','OPT_DISP_Q_12'],inplace=True,axis=1)
MERCHCAT_MBQ.drop(['ST_OPT_DISP_Q_1','ST_OPT_DISP_Q_2','ST_OPT_DISP_Q_3','ST_OPT_DISP_Q_4','ST_OPT_DISP_Q_5','ST_OPT_DISP_Q_6'],inplace=True,axis=1)
MERCHCAT_MBQ.drop(['ST_OPT_DISP_Q_7','ST_OPT_DISP_Q_8','ST_OPT_DISP_Q_9','ST_OPT_DISP_Q_10','ST_OPT_DISP_Q_11','ST_OPT_DISP_Q_12'],inplace=True,axis=1)


In [ ]:
MERCHCAT_MBQ['DISP_Q_1'] = pd.to_numeric(MERCHCAT_MBQ['DISP_Q_1'], errors='coerce')

In [ ]:
MAJCAT_MBQ_A = MAJCAT_MBQ[['ST_CD','MAJCAT','MJ_MBQ_A','MJ_MBQ_30_1','MJ_MBQ_30_2','MJ_MBQ_30_3','MJ_MBQ_30_4','MJ_MBQ_30_5',
                          'MJ_MBQ_30_6','MJ_MBQ_30_7','MJ_MBQ_30_8','MJ_MBQ_30_9','MJ_MBQ_30_10','MJ_MBQ_30_11','MJ_MBQ_30_12']]

In [ ]:
MERCHCAT_MBQ_A = MERCHCAT_MBQ[['ST_CD','MAJCAT','MERCHCAT','MC_MBQ_A','MC_MBQ_30_1','MC_MBQ_30_2','MC_MBQ_30_3','MC_MBQ_30_4','MC_MBQ_30_5',
                          'MC_MBQ_30_6','MC_MBQ_30_7','MC_MBQ_30_8','MC_MBQ_30_9','MC_MBQ_30_10','MC_MBQ_30_11','MC_MBQ_30_12']]

In [ ]:
MERCHCAT_MBQ_TEMP = MERCHCAT_MBQ[['ST_CD','MERCHCAT','OPT_MBQ_A','OPT_MBQ_L1_A','OPT_MBQ_30_A','SSN_MBQ_A']]

In [ ]:
def save_csv(df, filename):
    df.to_csv(filename, index=False)

In [ ]:
path = r"C:\Users\Administrator\Desktop\drive-download-20240503T040120Z-001\Testing"


# List of DataFrames and corresponding file paths
dataframes = [MAJCAT_MBQ, MERCHCAT_MBQ, MERCHCAT_MBQ_A, MAJCAT_MBQ_A]
filenames = [path + r"\MJ_MBQ_file.csv", path + r"\MC_MBQ_file.csv",
             path + r"\MC_MBQ_B_file.csv", path + r"\MJ_MBQ_B_file.csv"]

# List to hold threads
threads = []

# Create and start a thread for each DataFrame
for df, filename in zip(dataframes, filenames):
    thread = threading.Thread(target=save_csv, args=(df, filename))
    thread.start()
    threads.append(thread)

# Wait for all threads to complete
for thread in threads:
    thread.join()


In [ ]:
MJ_MBQ = MAJCAT_MBQ_A
MC_MBQ = MERCHCAT_MBQ_A

In [ ]:
ST_STK_DATA = ST_STK
ST_STK_DATA = pd.merge(ST_STK_DATA, MERCHCAT_MBQ_TEMP, how="left",on=["ST_CD","MERCHCAT"],suffixes=('_left', '_right'))

In [ ]:
ST_STK_DATA['REV_ST_STK'] = np.where(ST_STK_DATA['OPT_MBQ_A'] + 0 == 0,
                                     ST_STK_DATA['ST_STK'],
                                     np.minimum(ST_STK_DATA['ST_STK'], ST_STK_DATA['OPT_MBQ_A']))

In [ ]:
ST_STK_DATA_TEMP = ST_STK_DATA[['ST_CD','MERCHCAT','GEN_ART','COLOR','ST_STK','REV_ST_STK']]

In [ ]:
#REV STK BTM-UP AT ALL HIER LEVEL
ST_STK_MJ = ST_STK_DATA.groupby(['ST_CD','MAJCAT'], as_index=False).agg(MJ_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC = ST_STK_DATA.groupby(['ST_CD','MERCHCAT'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_M_MVGR = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MACRO_MVGR'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_MVGR_1 = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MVGR_1'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_FABRIC = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','FABRIC'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_BODY = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','BODY'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_COLOR = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','COLOR'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_M_MVGR_COLOR = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MACRO_MVGR','COLOR'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_MVGR_1_COLOR = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MVGR_1','COLOR'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_FABRIC_COLOR = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','FABRIC','COLOR'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_BODY_COLOR = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','BODY','COLOR'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_M_MVGR_MVGR_1 = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MACRO_MVGR','MVGR_1'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_M_MVGR_FABRIC = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MACRO_MVGR','FABRIC'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_M_MVGR_BODY = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MACRO_MVGR','BODY'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_M_MVGR_MVGR_1_COLOR = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MACRO_MVGR','MVGR_1','COLOR'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_M_MVGR_FABRIC_COLOR = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MACRO_MVGR','FABRIC','COLOR'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))
ST_STK_MC_M_MVGR_BODY_COLOR = ST_STK_DATA.groupby(['ST_CD','MERCHCAT','MACRO_MVGR','BODY','COLOR'], as_index=False).agg(MC_REV_STK=('REV_ST_STK','sum'))


In [ ]:
#download data ST_STK_DATA
temp=ST_STK_DATA
temp.to_csv(path+r"\ST_STK_temp_file.csv")


In [ ]:
DC_STK_DATA = DC_STK
ST_DC=DC_STK_DATA[['SEG','DIV','SUB_DIV','MAJCAT','MERCHCAT','MRP','RNG_SEG','SSN','GEN_ART','GEN_ART_DESC','COLOR','VAR_ART','SZ','M_VND_CD',
              'M_VND_NM','CO_AGING','MACRO_MVGR','MVGR_1','FABRIC','BODY','DISCOUNT_STATUS']]
ST_DC.insert(0,"ST_NO",0)
ST_DC['ST_NO']=0
ST_TEST = ST_DC
ST_DATA = ST_DC

In [ ]:
# Increment the 'ST_NO' column by 1 for values from 0 to 199
ST_TEST['ST_NO'] += 1

# Concatenate ST_TEST with ST_DATA
ST_DATA = pd.concat([ST_DATA] * 200, ignore_index=True)

# Update the 'ST_NO' column in the concatenated DataFrame
ST_DATA['ST_NO'] = np.tile(ST_TEST['ST_NO'], len(ST_DATA) // len(ST_TEST))


In [ ]:
DATACHECK = ST_DATA[(ST_DATA['ST_NO']==1)].index
ST_DATA.drop(DATACHECK, inplace=True)

In [ ]:
ST_DC['ST_NO']=1
ST_DATA = pd.concat([ST_DATA,ST_DC],ignore_index=True)

In [ ]:
ST_DETAILS_R = ST_DETAILS[['ST_CD','PRIORITY_LIST']]
ST_DETAILS_R['ST_NO']=ST_DETAILS_R['PRIORITY_LIST']
ST_DETAILS_R = ST_DETAILS[['ST_CD','ST_NO']]

In [ ]:
ST_DATA = pd.merge(ST_DATA, ST_DETAILS_R, how="left",on=["ST_NO"])

In [ ]:
ST_DATA['ST_CD'] = ST_DATA['ST_CD'].fillna("xxx")
DATACHECK = ST_DATA[(ST_DATA['ST_CD']=='xxx')].index
ST_DATA.drop(DATACHECK, inplace=True)

In [ ]:
ST_DATA = pd.merge(ST_DATA, MERCHCAT_MBQ_TEMP, how="left",on=["ST_CD","MERCHCAT"])
ST_DATA = pd.merge(ST_DATA, ST_STK_DATA_TEMP, how="left",on=["ST_CD","MERCHCAT","GEN_ART","COLOR"])
ST_DATA['ST_STK'] = ST_DATA['ST_STK'].fillna(0)
ST_DATA['REV_ST_STK'] = ST_DATA['REV_ST_STK'].fillna(0)

In [ ]:
temp=ST_DATA.head(10000)
temp.to_csv(path+r"\ST_DATA_temp_file.csv")

In [ ]:
#comb
DC_STK_DATA['REV_DC_STK']=DC_STK_DATA['CURR_DC_STK']
DC_STK_DATA['GEN_CLR'] = DC_STK_DATA['GEN_ART'].astype(str)+DC_STK_DATA['COLOR'].astype(str)
DC_STK_DATA['MC_GEN_CLR'] = DC_STK_DATA['MERCHCAT'].astype(str)+DC_STK_DATA['GEN_ART'].astype(str)+DC_STK_DATA['COLOR'].astype(str)

In [ ]:
ST_DATA['GEN_CLR'] = ST_DATA['GEN_ART'].astype(str)+ST_DATA['COLOR'].astype(str)
ST_DATA['COMB'] = ST_DATA['ST_CD']+ST_DATA['GEN_ART'].astype(str)+ST_DATA['COLOR'].astype(str)
ST_DATA['GEN_CLR'] = ST_DATA['GEN_ART'].astype(str)+ST_DATA['COLOR'].astype(str)
ST_DATA['COMB'] = ST_DATA['ST_CD']+ST_DATA['GEN_ART'].astype(str)+ST_DATA['COLOR'].astype(str)
ST_DATA['ART_REQ'] = np.where(ST_DATA['ST_STK'] == 0,
                               ST_DATA['OPT_MBQ_A'],
                               np.maximum(ST_DATA['OPT_MBQ_A'] - ST_DATA['ST_STK'], 0))

# Calculate 'SSN_ART_REQ' using vectorized operations
ST_DATA['SSN_ART_REQ'] = np.where(ST_DATA['ST_STK'] == 0,
                                   ST_DATA['SSN_MBQ_A'],
                                   np.maximum(ST_DATA['SSN_MBQ_A'] - ST_DATA['ST_STK'], 0))

In [ ]:
DC_STK_DATA['ALLOC']=0
ST_DATA['ALLOC']=0

ST_STK_MJ['MJ_ALLOC']=0
ST_STK_MC['MC_ALLOC']=0
ST_STK_MC_M_MVGR['ALLOC'] = 0
ST_STK_MC_MVGR_1['ALLOC'] = 0
ST_STK_MC_FABRIC['ALLOC'] = 0
ST_STK_MC_BODY['ALLOC'] = 0
ST_STK_MC_COLOR['ALLOC'] = 0
ST_STK_MC_M_MVGR_COLOR['ALLOC'] = 0
ST_STK_MC_MVGR_1_COLOR['ALLOC'] = 0
ST_STK_MC_FABRIC_COLOR['ALLOC'] = 0
ST_STK_MC_BODY_COLOR['ALLOC'] = 0
ST_STK_MC_M_MVGR_MVGR_1['ALLOC'] = 0
ST_STK_MC_M_MVGR_FABRIC['ALLOC'] = 0
ST_STK_MC_M_MVGR_BODY['ALLOC'] = 0
ST_STK_MC_M_MVGR_MVGR_1_COLOR['ALLOC'] = 0
ST_STK_MC_M_MVGR_FABRIC_COLOR['ALLOC'] = 0
ST_STK_MC_M_MVGR_BODY_COLOR['ALLOC'] = 0
MJ_MBQ['ALLOC'] = 0

In [ ]:
MJ_ALGO = BGT_MAJCAT[['ST_CD','MAJCAT']]
MC_ALGO = BGT_MERCHCAT[['ST_CD','MAJCAT','MERCHCAT']]
MC_M_MVGR = MC_M_MVGR_CONT[['ST_CD','MERCHCAT','MACRO_MVGR']]
MC_MVGR = MC_MVGR_CONT[['ST_CD','MERCHCAT','MVGR']]
MC_BODY = MC_BODY_CONT[['ST_CD','MERCHCAT','BODY']]
MC_FABRIC = MC_FABRIC_CONT[['ST_CD','MERCHCAT','FABRIC']]
MC_CLR = MC_CLR_CONT[['ST_CD','MERCHCAT','CLR']]
MC_VEND = MC_VEND_CONT[['ST_CD','MERCHCAT','VEND_CD']]


In [ ]:
MJ_MBQ['ALLOC']=MJ_MBQ['ALLOC'].astype(float)

In [ ]:

#MAJCAT_MBQ_A['COMB']= MAJCAT_MBQ_A['ST_CD']+MAJCAT_MBQ_A['MAJCAT']
MJ_MBQ['COMB'] = MJ_MBQ['ST_CD']+MJ_MBQ['MAJCAT']
MC_MBQ['COMB'] = MC_MBQ['ST_CD']+MC_MBQ['MERCHCAT']
#ST_STK_MJ['COMB'] = ST_STK_MJ['ST_CD']+ST_STK_MJ['MAJCAT']
#ST_STK_MC['COMB'] = ST_STK_MC['ST_CD']+ST_STK_MC['MERCHCAT']
MC_M_MVGR['COMB'] = MC_M_MVGR['ST_CD']+MC_M_MVGR['MERCHCAT']+MC_M_MVGR['MACRO_MVGR']
MC_MVGR['COMB'] = MC_MVGR['ST_CD']+MC_MVGR['MERCHCAT']+MC_MVGR['MVGR']
MC_BODY['COMB'] = MC_BODY['ST_CD']+MC_BODY['MERCHCAT']+MC_BODY['BODY']
MC_FABRIC['COMB'] = MC_FABRIC['ST_CD']+MC_FABRIC['MERCHCAT']+MC_FABRIC['FABRIC']
MC_CLR['COMB'] = MC_CLR['ST_CD']+MC_CLR['MERCHCAT']+MC_CLR['CLR']
MC_VEND['COMB'] = MC_VEND['ST_CD']+MC_VEND['MERCHCAT']+MC_VEND['VEND_CD']


In [ ]:
MJ_MBQ['MJ_MBQ']=MJ_MBQ['MJ_MBQ_A']
MC_MBQ['MC_MBQ']=MC_MBQ['MC_MBQ_A']
MJ_MBQ = pd.merge(MJ_MBQ, ST_STK_MJ, how="left",on=["ST_CD","MAJCAT"])
MC_MBQ = pd.merge(MC_MBQ, ST_STK_MC, how="left",on=["ST_CD","MERCHCAT"])


In [ ]:
print(MJ_MBQ.columns.to_list())

In [ ]:
MJ_MBQ['MJ_REQ'] = MJ_MBQ['MJ_MBQ']-(0.0+MJ_MBQ['MJ_REV_STK'])
MC_MBQ['MC_REQ'] = MC_MBQ['MC_MBQ']-(0.0+MC_MBQ['MC_REV_STK'])

In [ ]:
dataframes = [MJ_MBQ, MC_MBQ, ST_STK_MJ, ST_STK_MC, DC_STK_DATA]
file_paths = [path + r"\MJ_MBQ.csv", path + r"\MC_MBQ.csv", path + r"\MJ_STK.csv", 
              path + r"\MC_STK.csv", path + r"\DC_STK.csv"]

# Create a list to hold the threads
threads1 = []

# Iterate over each DataFrame and corresponding file path
for df, filename in zip(dataframes, file_paths):
    # Create a new thread for saving each DataFrame
    thread = threading.Thread(target=save_csv, args=(df, filename))
    thread.start()  # Start the thread
    threads1.append(thread)  # Add the thread to the list of threads

# Wait for all threads to complete
for thread in threads1:
    thread.join()

In [ ]:
A1=ST_DATA.columns
ST_DATA_NEW = pd.DataFrame(columns=A1)
A2=DC_STK_DATA.columns
DC_DATA_NEW = pd.DataFrame(columns=A2)
A3=MJ_MBQ.columns
MJ_MBQ_NEW = pd.DataFrame(columns=A3)
A4=MC_MBQ.columns
MC_MBQ_NEW = pd.DataFrame(columns=A4)


In [ ]:
DC_STK_DATA['ALLOC']=0
ST_DATA['ALLOC']=0
MJ_MBQ['ALLOC'] = 0
MC_MBQ['ALLOC'] = 0
MC_M_MVGR['ALLOC'] = 0
MC_MVGR['ALLOC'] = 0
MC_BODY['ALLOC'] = 0
MC_FABRIC['ALLOC'] = 0
MC_CLR['ALLOC'] = 0
MC_VEND['ALLOC'] = 0

In [ ]:
DC_STK_DATA['ALLOC']=0
ST_DATA['ALLOC']=0
MJ_MBQ['ALLOC'] = 0
MC_MBQ['ALLOC'] = 0
MC_M_MVGR['ALLOC'] = 0
MC_MVGR['ALLOC'] = 0
MC_BODY['ALLOC'] = 0
MC_FABRIC['ALLOC'] = 0
MC_CLR['ALLOC'] = 0
MC_VEND['ALLOC'] = 0

In [ ]:
#for i in MAJCAT_LIST1:
i = CAT
ST_DATA_TEMP = ST_DATA
# print(datetime.now())
for j in ST_DETAILS['ST_CD']:
    DC_STK_TEMP = DC_STK_DATA[(DC_STK_DATA['REV_DC_STK']>0)].reset_index(drop=True)
# MERCHCAT LEVEL LISTING
    for k in DC_STK_TEMP['GEN_CLR']:
        l = DC_STK_TEMP.loc[DC_STK_TEMP['GEN_CLR']==k,'MERCHCAT'].item()
        ART_REQ = ST_DATA_TEMP.loc[ST_DATA_TEMP['COMB'] == str(j)+str(k),'ART_REQ'].sum()
        #ART EXISTENCE IN ST-DATA & THERE SHOULD BE NO ALLOC IN PREVIOUS
        if DC_STK_TEMP.loc[DC_STK_TEMP['GEN_CLR']==k,'REV_DC_STK'].sum() >= ART_REQ:
            if ST_DATA_TEMP.loc[ST_DATA_TEMP['COMB'] == str(j)+str(k),'ART_REQ'].item()>0:
                if (MJ_MBQ.loc[MJ_MBQ['COMB'] == str(j)+str(i),'MJ_REQ'].sum()) >= ART_REQ:
                    if (MC_MBQ.loc[MC_MBQ['COMB'] == str(j)+str(l),'MC_MBQ'].sum()) >= ART_REQ:
                        #IF CONDITION TRUE THEN EXECUTE THE BELOW STEPS
                        ALLOC=0
                        ALLOC = ST_DATA_TEMP.loc[ST_DATA_TEMP['COMB'] == str(j)+str(k),'ART_REQ'].sum()
                        ST_DATA_TEMP.loc[ST_DATA_TEMP['COMB'] == str(j)+str(k),'ALLOC'] = ALLOC
                        ST_DATA_TEMP.loc[ST_DATA_TEMP['COMB'] == str(j)+str(k),'ART_REQ'] -= ALLOC
                        DC_STK_TEMP.loc[DC_STK_TEMP['GEN_CLR']==k,'ALLOC'] += ALLOC
                        DC_STK_TEMP.loc[DC_STK_TEMP['GEN_CLR']==k,'REV_DC_STK'] -= ALLOC
                        MJ_MBQ.loc[MJ_MBQ['COMB'] == str(j)+str(i),'ALLOC'] += ALLOC
                        MC_MBQ.loc[MC_MBQ['COMB'] == str(j)+str(l),'ALLOC'] += ALLOC
                        MJ_MBQ.loc[MJ_MBQ['COMB'] == str(j)+str(i),'MJ_REQ'] -= ALLOC
                        MC_MBQ.loc[MC_MBQ['COMB'] == str(j)+str(l),'MC_REQ'] -= ALLOC
    print(j)
    # print(datetime.now())

if ST_DATA_NEW is None:  
    ST_DATA_NEW = ST_DATA_TEMP
else:
    ST_DATA_NEW = pd.concat([ST_DATA_NEW,ST_DATA_TEMP],ignore_index=True)
if DC_DATA_NEW is None:  
    DC_DATA_NEW = DC_STK_TEMP
else:
    DC_DATA_NEW = pd.concat([DC_DATA_NEW,DC_STK_TEMP],ignore_index=True)
# print(datetime.now())

In [ ]:
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")


In [5]:
list = [i for i in range(0,10000000)]

In [ ]:
list=[]

for i in range(0,10000000):
    list.append(i)